//SPDX-License-Identifier: GPL-3.0-or-later

pragma solidity ^0.8.1;// 그냥 원하는 버전

pragma experimental ABIEncoderV2;// 실험용 기능 사용시



In [1]:
%%writefile SimpleChild.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Parent {
    //state variables
    address owner; //as of 0.8.0 no need to be address payable owner;
    uint private counter;
    
    //constructor. no need for constructor to be public as of 0.7.0
    constructor() {
        owner=msg.sender;
        counter = 0;
    }
    //functions
    function add() public { counter++; }
    function getCounter() public view returns(uint) { return counter; }
}

contract SimpleChild is Parent {// 상속
    //state variables
    string nickName;
    mapping(address => uint) private balances;// json같은 형식 -> mapping
    //event
    event PrintLog(address, uint);// 함수를 이벤트로 발생시켜 로그를 볼 수 있음
    
    //constructor
    constructor() {}  // no public as in constructor() public {}  
    //functions
    function setNickName(string memory s) public { nickName = s; }
    function getNickName() public view returns(string memory) { return nickName; }
    function deposit() public payable {// 결제시 사용
        balances[msg.sender] += msg.value;// value 값을 추가해줌
        emit PrintLog(msg.sender, msg.value);
    }
    function queryBalance() public view returns (uint) {
        return balances[msg.sender];// 잔고를 읽어옴
    }
    //access non-private members of the parent
    function kill() public {
        if (msg.sender == owner) selfdestruct(payable(owner)); //0.6.x selfdestruct(owner)
    }
}

Writing SimpleChild.sol


 Solidity에서 파일명은 컨트랙명과 일치하지 않아도 되는데, 파일에 하나 이상의 컨트랙트를 포함할 수 있기 때문에 그렇다. 파일명과 컨트랙명은 일치시키는 것을 권고한다.
* import문은 프로그램의 시작 부분에 작성하되, 프로그램 위, pragma 다음 줄에 적어준다.
* 컨트랙은 새로운 줄에 시작한다.
* 컨트랙, event, enum, struct은 낙타 표기법(camel case)을 따라 단어의 첫글자는 대문자로 작성한다.
* 함수, 함수인자, 변수, 함수 변경자(modifier)는 소문자로 시작하고 낙타 표기법으로 작성한다.
* 상수는 모두 대문자로 작성하되, 단어 사이를 밑줄 문자(underscore)로 연결해서 적어준다 (예: DATE_OF_BIRTH).
* 한 파일에 여러 컨트랙을 포함할 경우, 컨트랙 간에는 2줄 띄어쓰기를 해준다.
* 들여쓰기 할 때 탭 문자를 사용하지 말고 공백 문자 4칸을 넣어주자.
* 배열은 ```int[] x;```이라고 적어준다 (```int [] x``` 또는 ```int x[]```가 아니라)
* 문자열은 쌍따옴표로 표시한다. 
* 함수는 새로운 줄에 작성한다.
* 한 줄은 최대 79 문자를 넘지 않도록 하자. 80x25
* 괄호에서는 한 칸 띄어쓰기를 하지 않는다. ```if (x == 1)``` (```if ( x == 1 )```이 아니라)
* 연산자 앞 뒤에는 공백 1칸을 넣는다.
* 코드 블록을 적을 때, 다음과 같이 줄바꿈 없이 중괄호를 연결한다.

## 1.4 예약어

예약어(reserved word)는 키워드(keyword)라고 부르기도 하며 Solidity 언어에서 특별한 목적으로 사용하기 위해 지정한 단어들이다. 이런 명령어는 컨트랙이나 변수를 명명할 때 사용하지 않는다.

다음은 예약어를 알파벳 순으로 나열한 것이다.

abstract, after, alias, apply, auto, byte, case, catch, constant, copyof, default,
define, final, fixed, immutable, implements, import, in, inline, interface,
let, macro, match, mutable, null, of, override, partial, pragma, promise, reference, relocatable, returns, rollback, sealed, sizeof, static, supports, switch, try, type, 
typedef, typeof, using, var, virtual, weak, unchecked

Solidity 언어에는 더 많은 키워드가 있을 수 있으며, 새로운 버전이 출시될 때마다 업데이트 될 수 있다.

# 2. 데이터 타입

Solidity 언어는 공간이 제한된 블록체인에서 실행되는 까닭에 **저장공간을 효율적**으로 사용하는 편이 좋다. 저장공간을 많이 사용할수록 비용이 증가한다. - 변수 설정도 주의

다른 언어에서 지원하는 데이터타입을 큰 차이 없이 사용할 수 있지만, 메모리를 가급적 적게 사용할 수 있도록 설계되어 있어 불필요한 낭비를 줄이고 있다. 메모리는 실로 중요한 자원이고, 블록체인에서 불필요하게 또는 과도하게 ```배열을 검색```하거나 ```반복문```을 사용하는 것을 유의해야 한다.

그리고 소수점은 아직 지원되지 않아 float, double과 같은 자료형이 없다. 또한 암호화폐를 가지고 있어, 계정 주소 타입이 있다는 점이 특별나다.

 자료형    | 설명
----------|----------
bool | 내부적으로 uint8로 표현됨(0 또는 1을 나타냄). 거짓(false)은 0으로 참(true)은 1로 표현된다. 
uint<M> | 양의 정수(unsigned integer), M은 0 ~ 256비트까지, 0 < M <= 256, M은 8의 배수만 사용 가능. 
int<M> | 부호가 있는 정수(signed integer), M은 0 ~ 256비트까지 가능하다 0 < M <= 256, M은 8의 배수만 사용 가능. 
address | 주소. 크기가 20바이트이므로 **uint160** 이다.
uint, int | 숫자가 붙지 않은 경우의 타입으로 uint256, int256를 의미
bytes<M> | 바이너리 타잎, M은 32바이트까지 가능하다 0 < M <= 32. 크기를 지정하지 않는 **```bytes```는 value type이 아니다**.// 고정길이
string | UTF-8 문자열, 동적으로 크기가 정해지므로 **value type이 아니다**. //가변길이

bool isMarried=true;

int256 x=1;

소수 계산 지원 x OpenZeppelin의 SafeMath 라이브러리에 부동 소수점 연산

실행할때 solidity에서 변수형이 어떻게 받아오는가

bytes1 x= 0xFF;
bytes23 place1 = "7 hongji-dong jongro-gu"; //23글자에 맞추어 bytes23으로 선언
bytes8 place2 = "7 hongji"; //8글자에 맞추어 bytes8로 선언.

바이트는 문자열을 사용해도 되는데 알아서 바이트 형태로 바꿔줌

In [ ]:
# mvc
# model view controller 3계층 -> 덤터미널 -> 80줄???

In [3]:
%%writefile IntBool.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract IntBoolTest {
    bool married = true;
    uint256 xAge = 22;
    uint256 yAge = 25;
    //fixed phi; // = 3.14; // 선언만 가능
    function update() public {
        xAge = yAge;
        yAge = 33;
    }
    function setXAge(int age) public {
        xAge = uint(age);  //type conversion
    }
    function getXAge() public view returns(uint) {
        return xAge;
    }
    function getYAge() public view returns(uint) {
        return yAge;
    }
    function testInt() public view returns(bool) {
        assert(xAge>=20 && yAge>=20);
        return true;
    }
    function isMarried() public view returns(bool) {
        return married;
    }
}

Writing IntBool.sol


# byte

byte3 = "123"

static한 값
범위 1~32byte
기본 선언 byte는 byte1과 같다
hex값은 0x를 붙여서 작성 
문자열을 넣으면 hex값으로 저장됨
*hex는 hex값으로 두 글자가 1byte이다*
# bytes, string

string n = "hello"
string constant n = "hello"

string은 가변성을 가짐 그래서 bytes(n).length 이런 방식으로 바꿔서 값을 찾을 수 있음
가변성을 가짐 값을 보낼때 memory를 작성해야됨
크기: bytes(string).length
값 읽기: bytes(string)[2]
한글도 결과 잘 나옴

# struct

    struct StructTest{
        uint num;
        string name;
        bool inEnrolled;
    }
    
    // 선언은 일반적인 구조체와 비슷
    Student s1=Student(201911111,"jslim",true);// memory 타입이라는데 string이 있어서 그런듯
    Student s2;
    function setStudent2(uint n, string memory sn, bool e) public {
        s2.num = n;
        s2.name = sn;
        s2.isEnrolled = e;
    }
    
    // 반환 값은 값을 받는 쪽에서 stuct구조가 보통 없기 때문에 값을 하나씩 뽑아서 반환 해주는 것이 좋다
    function getStudent1() public view returns(uint, string memory, bool){
       return (s1.num, s1.name, s1.isEnrolled);   // 항목을 괄호로 묶어서 반환한다
    }
    *여기는 *
    function getStudentName() pure public returns(string memory) {
       // 우측 값이 함수 내에서 생성되기 때문에, 좌측을 memory를 사용한다고 해준다. storage는 사용 x
       Student memory s3 = Student(201911112, "jsl3", true);
       return s3.name;
    }

# enumeration

'집합' 요소에 있는 값만 사용이 가능하다 ex) 요일, 성별

enum Gender {male, female}

Gender man = Gender.male;// index[0]

    // 반환 받는 곳에서 Gender라는 값이 없다는 것을 전제로 생각하면 편하다

    function getMyDay() public view returns(Day) {
        return myDay;// return 값은 index값이 나오게 된다
    }
    /* @param 인자 d는 정수 uint8이면 된다*/
    function setMyDay(Day d) public {
        myDay = d;// 0 or 1을 넣어서 값을 할당 그럼 Day[0 or 1]로 들어감
    }
    //uint를 넘겨주어도 기본 uint8로 변환된다
    function setMyDayInt(uint d) public {
        myDay = Day(d); // 여기는 값을 받아서 0 or 1을 Day에 넣어줌
    }

# array

정적, 동적 배열 둘다 가능
uint[5] arrayName = [1,2,3,4,5];
int[] arrayName2;

    function initArray2 public {
        arrayName2 = new int[](5);// 배열의 크기를 할당
    }
    function appendMark(int mark) public {
        marks.push(mark);
    }
    function popMark() public {
        marks.pop();
    }
    /* @return 동적 배열을 반환하는 경우 memory를 사용한다*/
    function getMarks() public view returns(int[] memory) {
        return marks;
    }
    /* @return 고정 배열을 반환하는 경우 memory를 사용한다*/
    function getAges() public view returns(uint[3] memory) {
        return ages;
    }
    *이건 좀 헷갈림*
    function getLenOfArr() pure public returns(uint) {
        //memory is used because locally created array is assigned
        uint8[3] memory intArr = [0, 1, 2];
        return intArr.length;
    }

# address

20 byte길이의 객체

입출금 할 시 address payable로 선언 필요
단위는 wei
차이점 | send() | transfer() |  call.value()
-----|-----|-----|-----
gas 수정가능 | N | N | Y
gas 한도 | 2300 | 2300 | 모든 가용 gas를 넘겨주거나, 일정 gas를 정할 수 있다.
실패하면  | false | throw 예외발생 | false

송금이 실패하는 이유에는 여러 가지가 있을 수 있다. 
```call statck error``` 또는 ```out of gas error``` 등의 이유로 실패하는 경우에도  **예외(Exception)가 발생하지 않고 false를 반환**하게 된다.

In [1]:
%%writefile addressTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract AddressTest {
    address owner;
    address payable receiver; // 수신 계정;
    uint balanceOfOwner;
    
    // 송신자 계정 설정
    constructor() { // NO! constructor() public {
        owner=msg.sender;// 전송자의 주소
        balanceOfOwner = owner.balance; // 잔고 설정
    }
    // *? ?*
    function deposit() payable public {
        // payable로 선언해서, 코드가 없어도 owner에게 입금이 된다.
    }
    
    // 계정 설정
    /* @param 수신 계정을 설정*/
    function setReceiver(address payable addr) public {
        receiver=addr;
    }
    function getReceiver() view public returns(address) {
        return receiver;
    }

    // 잔고를 보는 방법 3가지
    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;  // 컨트랙 자체의 잔고
    }
    function getBalanceOfOwner() public view returns(uint) {
        return owner.balance;
    }
    function getBalanceOfReceiver() public view returns(uint) {
        return receiver.balance;
    }

    function send() public payable {
        require(receiver.send(111)); // 111 wei를 receiver로 송금. require()로 예외처리. - boolean값만 가능
    }
    function transfer() public payable {
        //if !(receiver.transfer(address(this).balance))
        receiver.transfer(11111); // receiver로 송금
    }
    function callValue() public payable {
        //receiver.call.value(11111)(""); // 과거 문법. 이렇게 작성하지 않는다.
        (bool success, ) = receiver.call{value: 11111}(""); //gas를 적어주지 않으면 실제 사용량 지급
        require(success, "transfer call failed.");
        (success, ) = receiver.call{gas: 10, value: 11111}(""); //gas를 적게 설정하고 테스트
        require(success, "transfer call failed with gas 10.");
    }
}

Writing addressTest.sol


# storage

32byte 길이의 key-value 형식

contract의 모든 함수에서 값을 볼 수 있음

# memory

지역변수 단기간 저장하는 경우에 사용

* 상태변수는 영구적인 저장소인 storage에 저장된다.
* 함수에 대해서 말하면, value 타입의 지역변수, 함수인자/반환은 memory를 기본 default로 사용한다.
단, 함수에서의 참조타입 (struct, array, mapping, string)은 기본적으로 storage에 저장되고, memory를 사용하려면 명시적으로 선언해야 한다.

# 가시성

구분 | 설명 | default
-----|-----|-----
```public``` | 누구나 ```internal``` 또는 ```external``` 모두 사용 | no
```private``` |  컨트랙 자신만 사용 | no
```internal``` | 현재 컨트랙 내부에서만 또는 상속의 경우에 사용하는 경우. java의 ```protected```와 비슷한 의미// 선언 안 하면 기본 값 | yes
```external``` | 외부에서 호출하는 경우만 허용 | no

# 연산자

* 산술연산자: +  - ```*```  /  ```%``` (modulus) ++ -- ```**``` (제곱함수)
* 비교연산자: ```==``` ```!=``` ```>``` ```<``` ```>=``` ```<=```
* 논리연산자: ```&&``` (논리AND) ```||``` (논리OR) ```!``` (논리NOT)
* 비트연산자: ```&``` (비트AND) ```|``` (비트OR) ```^``` (비트XOR) ```~``` (비트NOT) ```<<``` (비트왼쪽시프트) ```>>``` (비트오른쪽시프트)
* 대입연산자: ```=``` ```+=``` ```-=``` ```*=``` ```/=``` ```%=``` (```x = x % y```는 ```x %= y```)
* 삼항연산자: ```?:```

In [2]:
%%writefile myBank.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract MyBank {
    address owner; //address payable owner;
    uint balance;
    constructor() { //constructor() public {
        owner = msg.sender;
        balance = address(this).balance;// 이 잔고에서 보내는 거 인가봄
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);// 입금되는 값 == 입금 할 값
        balance += amount;
    }

    // 여기가 이해 안 됨
    function withdraw(uint amount) public payable {
        balance -= amount;   // 송금 전 감액
        payable(owner).transfer(amount); // owner에게 전송 하는 것 이건 직접 해보자
    }
    function transferTo(address payable receiver, uint amount) public payable {
        balance -= amount;   // 송금 전 감액
        receiver.transfer(amount);
    }

    function getBalance() public view returns (uint) {
        return balance;
    }
    function getBalanceOfThis() public view returns (uint) {
        return address(this).balance;
    }
    function getBalanceOfOwner() public view returns (uint) {
        return owner.balance;
    }
}

Writing myBank.sol


In [1]:
%%writefile DataConversionTest.sol
# conversion
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract DataConversionTest {
    function convertInt8ToInt() pure public returns(int) {
        int8 i8=20;
        return int(i8);  //20
    }
    function convertIntToBytes32() pure public returns(bytes32) {
        uint x=20;
        //0x0000000000000000000000000000000000000000000000000000000000000014
        return bytes32(x);
    }
    function convertInt64ToBytes8() pure public returns(bytes8) {
        uint64 x=10;
        return bytes8(x); //0x000000000000000a
    }

    // 바로 uint -> bytes는 불가
    // uint -> bytes32 -> bytes

    function convertIntToBytes() pure public returns(bytes memory) {
        // can not convert uint -> bytes
        // convert uint -> bytes32 -> bytes
        uint x = 20;
        bytes32 y = bytes32(x); //uint <=> uint256
        // can not convert from bytes32 -> bytes;
        bytes memory z = new bytes(32);
        for (uint i=0; i < 32; i++) {
            z[i] = y[i];
        }
        //0x0000000000000000000000000000000000000000000000000000000000000014
        return z;
    }

    function convertStringToBytes() pure public returns(bytes memory) {
        string memory s="hello";
        return bytes(s);  //0x68656c6c6f
    }
    function convertBytes4ToBytes2() pure public returns(bytes2) {
        bytes4 b4=0x68656c6c;
        return bytes2(b4);  //0x6865  overflow로 인한 데이터 손실
    }
    function convertBytes4ToInt32() pure public returns(uint32) {    
        bytes4 b4=0x68656c6c;
        //1751477356 = 68656C6C hex = (6 × 16^7) + (8 × 16^6) + ... + (6 × 16^1) + (12 × 16^0) 
        return uint32(b4); //1751477356 16진수를 10진수로 변환
    }
}


Writing DataConversionTest.sol


# Global variable

화폐 단위는 ```wei```, ```gwei```, ```ether```가 있다.  ```wei```는 1, 1 ```gwei```는 $10^{9}$ wei이고  1 ```ether```는 $10^{18}$ wei이다.

```
1 ether == 1000000000000000000 wei
1 gwei == 1000000000 wei

1 minutes == 60 seconds //1칸 띄워서 시간 단위를 붙여 쓴다.
1 hours == 60 minutes
1 days == 24 hours
1 weeks = 7 days
uint mytime=block.timestamp  //현재 시간을 저장. now는 더 이상 쓰지 못한다.

```block```은 블록의 속성들을 나타낸다.

* ```block.coinbase```: 현재 블록 마이너의 주소
* ```block.difficulty```: 현재 블록의 난이도
* ```block.gaslimit```: 현재 블록의 gaslimit
* ```block.number```: 현재 블록 수
* ```block.blockhash```: 현재 블록 해쉬 값
* ```block.timestamp```: 현재 블록 타임스탬프 epoch (1970년 1월 1일 0시) 이후 지나간 초, uint256 타입이다.

```tx```는 컨트랙을 호출하는 트랜잭션 관련 정보들을 가지고 있다. 

* ```tx.origin``` 트랜잭션에 사인한 계정
* ```tx.gasprice``` 트랜잭션 호출자가 명시한 gas price

```msg```는 컨트랙의 함수를 호출한 전송 관련 정보를 포함한다.

* ```msg.data```: call 데이터 (bytes 값으로 표현)
* ```msg.gas```: gas 분량
* ```msg.sender```: 현재 msg를 전달하는 측 주소
* ```msg.value```: 지급되는 ether (단위는 wei)

얼핏 보면, ```tx.origin```과 ```msg.sender``` 구별을 혼돈할 수 있다. 최초사용자 U1 -> 컨트랙 C1 -> 컨트랙 C2의 순서대로 호출된다고 하자.

* C2에서 ```msg.sender```는 바로 직전의 호출자C1을 말한다. 컨트랙도 ```msg.sender```가 될 수 있다.
* 반면에 ```tx.origin```은 U1을 의미한다. 최초로 메시지를 발생한 측을 말한다.